In [ ]:
import multirex as mrex
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import  ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
## general metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# accuracy, f1 , recall, precision
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
## hamming loss and exact match
from sklearn.metrics import hamming_loss, accuracy_score

%matplotlib inline

#load txt waves.txt
waves=np.loadtxt("waves.txt")
n_points = len(waves)
indices = np.linspace(0, len(waves) - 1, n_points, endpoint=True)
indices = np.round(indices).astype(int)  # Redondear los índices y convertir a entero

# Seleccionar los elementos de la lista usando los índices
puntos_seleccionados = waves[indices]
waves=puntos_seleccionados
wn_grid=np.sort((10000/waves))

In [ ]:
# Función para normalizar por filas con Min-Max, con manejo de casos especiales
def normalize_min_max_by_row(df):
    min_by_row = df.min(axis=1)
    max_by_row = df.max(axis=1)
    # Diferencia entre máximos y mínimos
    range_by_row = max_by_row - min_by_row
    # Evitar división por cero asignando NaN, que luego reemplazaremos
    normalized = (df.sub(min_by_row, axis=0)).div(range_by_row, axis=0)
    # Manejar filas donde max == min (rango es cero), puedes ajustar este valor
    normalized[range_by_row == 0] = 0  # O cualquier otro valor que consideres apropiado
    return normalized

import ast
# Función para convertir string a lista
def string_to_list(string):
    return ast.literal_eval(string)

def plot_confusion_matrix(y_test, y_pred, labels, display_labels, title):
    """
    Función para calcular y mostrar la matriz de confusión con etiquetas descriptivas y un título personalizado.

    Parámetros:
    - y_test: Array con los valores reales.
    - y_pred: Array con las predicciones del modelo.
    - labels: Lista de las etiquetas de clase en el orden en que deben ser tratadas.
    - display_labels: Lista con los nombres descriptivos para las etiquetas de las clases para visualización.
    - title: String con el título para el gráfico de la matriz de confusión.
    """

    # Calcula la matriz de confusión con las etiquetas descriptivas
    cm = confusion_matrix(y_test, y_pred, labels=labels, normalize='true')

    # Crea el display de la matriz de confusión con etiquetas descriptivas
    fig, ax = plt.subplots(figsize=(8, 8))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
    disp.plot(values_format=".0%", colorbar=True, ax=ax)
    disp.im_.colorbar.remove()  # Remueve la barra de color automática para reemplazarla
    disp.im_.set_clim(0, 1)  # Asegura que los límites de la barra de colores sean de 0 a 1

    # Crea manualmente la barra de colores con los límites correctos
    cbar = fig.colorbar(disp.im_, ax=ax)
    cbar.ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))  # Corrige el formateador para mostrar en porcentajes

    # Ajustes de formato y título
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
    plt.title(title, fontsize=20)

    plt.show()

## load data

In [ ]:
## load csv
CO2_data=pd.read_csv("spec_data/CO2_data.csv")
CO2_data.data=CO2_data.iloc[:,-n_points:]
CO2_data.params=CO2_data.iloc[:,:-n_points]

CH4_data=pd.read_csv("spec_data/CH4_data.csv")
CH4_data.data=CH4_data.iloc[:,-n_points:]
CH4_data.params=CH4_data.iloc[:,:-n_points]

O3_data=pd.read_csv("spec_data/O3_data.csv")
O3_data.data=O3_data.iloc[:,-n_points:]
O3_data.params=O3_data.iloc[:,:-n_points]

H2O_data=pd.read_csv("spec_data/H2O_data.csv")
H2O_data.data=H2O_data.iloc[:,-n_points:]
H2O_data.params=H2O_data.iloc[:,:-n_points]

CH4_O3_data=pd.read_csv("spec_data/CH4_O3_data.csv")
CH4_O3_data.data=CH4_O3_data.iloc[:,-n_points:]
CH4_O3_data.params=CH4_O3_data.iloc[:,:-n_points]

CH4_H2O_data=pd.read_csv("spec_data/CH4_H2O_data.csv")
CH4_H2O_data.data=CH4_H2O_data.iloc[:,-n_points:]
CH4_H2O_data.params=CH4_H2O_data.iloc[:,:-n_points]

O3_H2O_data=pd.read_csv("data/O3_H2O_data.csv")
O3_H2O_data.data=O3_H2O_data.iloc[:,-n_points:]
O3_H2O_data.params=O3_H2O_data.iloc[:,:-n_points]

CH4_O3_H2O_data=pd.read_csv("spec_data/CH4_O3_H2O_data.csv")
CH4_O3_H2O_data.data=CH4_O3_H2O_data.iloc[:,-n_points:]
CH4_O3_H2O_data.params=CH4_O3_H2O_data.iloc[:,:-n_points]

## Train data

### SNR = 1

In [ ]:
SNR1_CO2_data = mrex.generate_df_SNR_noise(
    df=CO2_data,
    n_repeat=20_000,
    SNR=1     
)

SNR1_CH4_data = mrex.generate_df_SNR_noise(
    df=CH4_data,
    n_repeat=500,
    SNR=1     
)

SNR1_O3_data = mrex.generate_df_SNR_noise(
    df=O3_data,
    n_repeat=300,
    SNR=1     
)

SNR1_H2O_data = mrex.generate_df_SNR_noise(
    df=H2O_data,
    n_repeat=1000,
    SNR=1     
)

SNR1_CH4_O3_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_data,
    n_repeat=20,
    SNR=1
)

SNR1_CH4_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_H2O_data,
    n_repeat=20,
    SNR=1
)

SNR1_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=O3_H2O_data,
    n_repeat=20,
    SNR=1
)

SNR1_CH4_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_H2O_data,
    n_repeat=5,
    SNR=1
)

    
SNR1_df = pd.concat(
    [
        SNR1_CO2_data,
        SNR1_CH4_data,
        SNR1_O3_data,
        SNR1_H2O_data,
        SNR1_CH4_O3_data,
        SNR1_CH4_H2O_data,
        SNR1_O3_H2O_data,
        SNR1_CH4_O3_H2O_data
    ],
    ignore_index=True
) 

### SNR = 3

In [ ]:
SNR3_CO2_data = mrex.generate_df_SNR_noise(
    df=CO2_data,
    n_repeat=20_000,
    SNR=3     
)

SNR3_CH4_data = mrex.generate_df_SNR_noise(
    df=CH4_data,
    n_repeat=500,
    SNR=3     
)

SNR3_O3_data = mrex.generate_df_SNR_noise(
    df=O3_data,
    n_repeat=300,
    SNR=3     
)

SNR3_H2O_data = mrex.generate_df_SNR_noise(
    df=H2O_data,
    n_repeat=1000,
    SNR=3     
)

SNR3_CH4_O3_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_data,
    n_repeat=20,
    SNR=3
)

SNR3_CH4_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_H2O_data,
    n_repeat=20,
    SNR=3
)

SNR3_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=O3_H2O_data,
    n_repeat=20,
    SNR=3
)

SNR3_CH4_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_H2O_data,
    n_repeat=5,
    SNR=3
)
    

SNR3_df = pd.concat(
    [
        SNR3_CO2_data,
        SNR3_CH4_data,
        SNR3_O3_data,
        SNR3_H2O_data,
        SNR3_CH4_O3_data,
        SNR3_CH4_H2O_data,
        SNR3_O3_H2O_data,
        SNR3_CH4_O3_H2O_data
    ],
    ignore_index=True
) 

### SNR = 6

In [ ]:
SNR6_CO2_data = mrex.generate_df_SNR_noise(
    df=CO2_data,
    n_repeat=20_000,
    SNR=6     
)

SNR6_CH4_data = mrex.generate_df_SNR_noise(
    df=CH4_data,
    n_repeat=500,
    SNR=6     
)

SNR6_O3_data = mrex.generate_df_SNR_noise(
    df=O3_data,
    n_repeat=300,
    SNR=6     
)

SNR6_H2O_data = mrex.generate_df_SNR_noise(
    df=H2O_data,
    n_repeat=1000,
    SNR=6     
)

SNR6_CH4_O3_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_data,
    n_repeat=20,
    SNR=6
)

SNR6_CH4_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_H2O_data,
    n_repeat=20,
    SNR=6
)

SNR6_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=O3_H2O_data,
    n_repeat=20,
    SNR=6
)

SNR6_CH4_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_H2O_data,
    n_repeat=5,
    SNR=6
)
    

SNR6_df = pd.concat(
    [
        SNR6_CO2_data,
        SNR6_CH4_data,
        SNR6_O3_data,
        SNR6_H2O_data,
        SNR6_CH4_O3_data,
        SNR6_CH4_H2O_data,
        SNR6_O3_H2O_data,
        SNR6_CH4_O3_H2O_data
    ],
    ignore_index=True
)

### SNR = 10

In [ ]:
### SNR = 10

SNR10_CO2_data = mrex.generate_df_SNR_noise(
    df=CO2_data,
    n_repeat=10_000,
    SNR=10     
)

SNR10_CH4_data = mrex.generate_df_SNR_noise(
    df=CH4_data,
    n_repeat=500,
    SNR=10    
)

SNR10_O3_data = mrex.generate_df_SNR_noise(
    df=O3_data,
    n_repeat=300,
    SNR=10    
)

SNR10_H2O_data = mrex.generate_df_SNR_noise(
    df=H2O_data,
    n_repeat=500,
    SNR=10    
)

SNR10_CH4_O3_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_data,
    n_repeat=20,
    SNR=10
)

SNR10_CH4_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_H2O_data,
    n_repeat=20,
    SNR=10
)

SNR10_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=O3_H2O_data,
    n_repeat=20,
    SNR=10
)

SNR10_CH4_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_H2O_data,
    n_repeat=5,
    SNR=10
)
    

SNR10_df = pd.concat(
    [
        SNR10_CO2_data,
        SNR10_CH4_data,
        SNR10_O3_data,
        SNR10_H2O_data,
        SNR10_CH4_O3_data,
        SNR10_CH4_H2O_data,
        SNR10_O3_H2O_data,
        SNR10_CH4_O3_H2O_data
    ],
    ignore_index=True
) 

### SNR = 20

In [ ]:
SNR20_CO2_data = mrex.generate_df_SNR_noise(
    df=CO2_data,
    n_repeat=10_000,
    SNR=20     
)

SNR20_CH4_data = mrex.generate_df_SNR_noise(
    df=CH4_data,
    n_repeat=500,
    SNR=20     
)

SNR20_O3_data = mrex.generate_df_SNR_noise(
    df=O3_data,
    n_repeat=300,
    SNR=20     
)

SNR20_H2O_data = mrex.generate_df_SNR_noise(
    df=H2O_data,
    n_repeat=500,
    SNR=20     
)

SNR20_CH4_O3_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_data,
    n_repeat=20,
    SNR=20
)

SNR20_CH4_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_H2O_data,
    n_repeat=20,
    SNR=20
)

SNR20_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=O3_H2O_data,
    n_repeat=20,
    SNR=20
)

SNR20_CH4_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_H2O_data,
    n_repeat=5,
    SNR=20
)
    

SNR20_df = pd.concat(
    [
        SNR20_CO2_data,
        SNR20_CH4_data,
        SNR20_O3_data,
        SNR20_H2O_data,
        SNR20_CH4_O3_data,
        SNR20_CH4_H2O_data,
        SNR20_O3_H2O_data,
        SNR20_CH4_O3_H2O_data
    ],
    ignore_index=True
) 

### SNR = NaN

In [ ]:
SNRnan_CO2_data = mrex.generate_df_SNR_noise(
    df=CO2_data,
    n_repeat=10000,
    SNR=10_000     
)

SNRnan_CH4_data = mrex.generate_df_SNR_noise(
    df=CH4_data,
    n_repeat=500,
    SNR=10_000     
)

SNRnan_O3_data = mrex.generate_df_SNR_noise(
    df=O3_data,
    n_repeat=500,
    SNR=10_000     
)

SNRnan_H2O_data = mrex.generate_df_SNR_noise(
    df=H2O_data,
    n_repeat=500,
    SNR=10_000     
)

SNRnan_CH4_O3_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_data,
    n_repeat=50,
    SNR=10_000
)

SNRnan_CH4_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_H2O_data,
    n_repeat=50,
    SNR=10_000
)

SNRnan_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=O3_H2O_data,
    n_repeat=50,
    SNR=10_000
)

SNRnan_CH4_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_H2O_data,
    n_repeat=5,
    SNR=10_000
)
    

SNRnan_df = pd.concat(
    [
        SNRnan_CO2_data,
        SNRnan_CH4_data,
        SNRnan_O3_data,
        SNRnan_H2O_data,
        SNRnan_CH4_O3_data,
        SNRnan_CH4_H2O_data,
        SNRnan_O3_H2O_data,
        SNRnan_CH4_O3_H2O_data
    ],
    ignore_index=True
) 

In [ ]:
SNRall= pd.concat([SNR1_df,SNR3_df,SNR6_df,SNR10_df,SNR20_df,SNRnan_df],
                  ignore_index=True)

SNRall["label"] = SNRall["label"].apply(string_to_list)
SNRall_train = SNRall

## normalize 
SNRall_train.iloc[:,-n_points:]=normalize_min_max_by_row(SNRall_train.iloc[:,-n_points:])

SNRall_X_train = SNRall_train.iloc[:,-n_points:]
SNRall_Y_train = np.array(SNRall_train["label"].tolist())


label
0    405000
1    375000
Name: count, dtype: int64

In [ ]:
rf= RandomForestClassifier(n_estimators=100,
                           max_depth= 200,
                           criterion= "entropy",
                           min_samples_leaf= 3,
                           n_jobs=-1,
                           )

rf.fit(SNRall_X_train,SNRall_Y_train)

## Test

In [ ]:
SNR = 1

SNR_test_CO2_data = mrex.generate_df_SNR_noise(
    df=CO2_data,
    n_repeat=10_000,
    SNR= SNR    
)

SNR_test_CH4_data = mrex.generate_df_SNR_noise(
    df=CH4_data,
    n_repeat=500,
    SNR= SNR    
)

SNR_test_O3_data = mrex.generate_df_SNR_noise(
    df=O3_data,
    n_repeat=500,
    SNR= SNR    
)

SNR_test_H2O_data = mrex.generate_df_SNR_noise(
    df=H2O_data,
    n_repeat=500,
    SNR= SNR    
)

SNR_test_CH4_O3_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_data,
    n_repeat=50,
    SNR=SNR
)

SNR_test_CH4_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_H2O_data,
    n_repeat=50,
    SNR=SNR
)

SNR_test_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=O3_H2O_data,
    n_repeat=50,
    SNR=SNR
)

SNR_test_CH4_O3_H2O_data = mrex.generate_df_SNR_noise(
    df=CH4_O3_H2O_data,
    n_repeat=10,
    SNR=SNR
)
    

SNR_test_df = pd.concat(
    [
        SNR_test_CO2_data,
        SNR_test_CH4_data,
        SNR_test_O3_data,
        SNR_test_H2O_data,
        SNR_test_CH4_O3_data,
        SNR_test_CH4_H2O_data,
        SNR_test_O3_H2O_data,
        SNR_test_CH4_O3_H2O_data
    ],
    ignore_index=True
) 
SNR_test_df["label"] = SNR_test_df["label"].apply(string_to_list)

SNR_test_df.iloc[:,-n_points:]=normalize_min_max_by_row(SNR_test_df.iloc[:,-n_points:])
SNR_X_test = SNR_test_df.iloc[:,-n_points:]
SNR_test_CH4 = np.array([item[0] for item in SNR_test_df["label"]]) 
SNR_test_O3 = np.array([item[1] for item in SNR_test_df["label"]])
SNR_test_H2O = np.array([item[2] for item in SNR_test_df["label"]])

SNR_test_df["CH4 label"] = SNR_test_CH4
SNR_test_df["O3 label"] = SNR_test_O3
SNR_test_df["H2O label"] = SNR_test_H2O

In [ ]:
CH4_probabilities= rf.predict_proba(SNR_X_test)[0][:,1]
O3_probabilities= rf.predict_proba(SNR_X_test)[1][:,1]
H2O_probabilities= rf.predict_proba(SNR_X_test)[2][:,1]
 #Asignar etiquetas basadas en el umbral ajustado
CH4_threshold = 0.49
O3_threshold = 0.46
H2O_threshold = 0.5


CH4_y_pred = (CH4_probabilities > CH4_threshold).astype(int)
O3_y_pred = (O3_probabilities > O3_threshold).astype(int)
H2O_y_pred = (H2O_probabilities > H2O_threshold).astype(int)

# Asignar etiquetas y puntuaciones a las predicciones
SNR_test_df["CH4 pred"] = CH4_y_pred
SNR_test_df["CH4 score"] = CH4_probabilities
SNR_test_df["O3 pred"] = O3_y_pred
SNR_test_df["O3 score"] = O3_probabilities
SNR_test_df["H2O pred"] = H2O_y_pred
SNR_test_df["H2O score"] = H2O_probabilities


## general metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# accuracy, f1 , recall, precision
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
## hamming loss and exact match
from sklearn.metrics import hamming_loss, accuracy_score

## prints
print("GLOBAL SCORES")
print("Hamming Loss")
print(hamming_loss(SNR_test_df[["CH4 label", "O3 label", "H2O label"]],
                   SNR_test_df[["CH4 pred", "O3 pred", "H2O pred"]]))
print("Exact Match")
print(accuracy_score(SNR_test_df[["CH4 label", "O3 label", "H2O label"]], 
                     SNR_test_df[["CH4 pred", "O3 pred", "H2O pred"]]))

print("Precision")
print(precision_score(SNR_test_df[["CH4 label", "O3 label", "H2O label"]], 
                      SNR_test_df[["CH4 pred", "O3 pred", "H2O pred"]], 
                      average="macro"))
print("Recall")
print(recall_score(SNR_test_df[["CH4 label", "O3 label", "H2O label"]],
                   SNR_test_df[["CH4 pred", "O3 pred", "H2O pred"]], 
                   average="macro"))
print("F1")
print(f1_score(SNR_test_df[["CH4 label", "O3 label", "H2O label"]],
               SNR_test_df[["CH4 pred", "O3 pred", "H2O pred"]],
               average="macro"))



# Imprimir la matriz de confusión y el informe de clasificación
print("CH4 SCORES")
print(confusion_matrix(SNR_test_CH4, CH4_y_pred))
print(classification_report(SNR_test_CH4, CH4_y_pred))
print("\n",10*"-")
print("O3 SCORES")
print(confusion_matrix(SNR_test_O3, O3_y_pred))
print(classification_report(SNR_test_O3, O3_y_pred))
print("\n",10*"-")
print("H2O SCORES")
print(confusion_matrix(SNR_test_H2O, H2O_y_pred))
print(classification_report(SNR_test_H2O, H2O_y_pred))


plot_confusion_matrix(
    SNR_test_CH4, CH4_y_pred,
    labels=[1,0], display_labels=["$CH_4$", "No $CH_4$"],
    title=f"Confusion Matrix for $CH_4$ Detection\n SNR= {SNR} "
)

plot_confusion_matrix(
    SNR_test_O3, O3_y_pred,
    labels=[1,0], display_labels=["$O_3$", "No $O_3$"],
    title=f"Confusion Matrix for $O_3$ Detection\n SNR= {SNR} "
)

plot_confusion_matrix(
    SNR_test_H2O, H2O_y_pred,
    labels=[1,0], display_labels=["$H_2O$", "No $H_2O$"],
    title=f"Confusion Matrix for $H_2O$ Detection\n SNR= {SNR} "
)


In [ ]:
CH4_SNR_test_by_atmCH4=[[] for i in range(10)]
mix_ratios = np.logspace(-10,-1,10)
for i, j in enumerate(range(-10,-0)):
    CH4_SNR_test_by_atmCH4[i]=SNR_test_df[SNR_test_df["atm CH4"]==j]
    #concatenate  CO2 spectra to each atm CH4
    co2=int(len(SNR_test_df[SNR_test_df["CH4 label"]==0])/10)
    no_CH4 = (
        #SNR_test_df[SNR_test_df["CH4 label"]==0][i*co2:(i+1)*co2]
        SNR_test_df[SNR_test_df["CH4 label"]==0].sample(co2)
        )
    CH4_SNR_test_by_atmCH4[i]=pd.concat([CH4_SNR_test_by_atmCH4[i],
                                            no_CH4])
    
    
CH4_SNR_f1 = np.zeros(len(mix_ratios))
CH4_SNR_recall = np.zeros(len(mix_ratios))
CH4_SNR_precision = np.zeros(len(mix_ratios))
CH4_SNR_score = np.zeros(len(mix_ratios))

for j in range(len(mix_ratios)):
                
                CH4_SNR_f1[j] = f1_score(
                    CH4_SNR_test_by_atmCH4[j]["CH4 label"].astype(int),
                    CH4_SNR_test_by_atmCH4[j]["CH4 pred"].astype(int)
                    )
                
                CH4_SNR_recall[j] = recall_score(
                    CH4_SNR_test_by_atmCH4[j]["CH4 label"].astype(int),
                    CH4_SNR_test_by_atmCH4[j]["CH4 pred"].astype(int)
                    )
                
                CH4_SNR_precision[j]=precision_score(
                    CH4_SNR_test_by_atmCH4[j]["CH4 label"].astype(int),
                    CH4_SNR_test_by_atmCH4[j]["CH4 pred"].astype(int)
                    )
## plot confusion matrix
fig, ax = plt.subplots(2, 5, figsize=(20, 10))
for i in range(len(cmatrix_plot)):
    # Plotea cada matriz de confusión en su respectivo subplot
    cmatrix_plot[i].plot(ax=ax[i//5, i%5], xticks_rotation='vertical', colorbar=False, values_format=".0%")
    ax[i//5, i%5].set_title(f"mix ratio: $CH_4$={mix_ratios[i]:.0e}")
    # Establece el límite de la barra de color a 0-1
    cmatrix_plot[i].im_.set_clim(0, 1)
    
fig.tight_layout(w_pad=3)
# Añade una barra de color común para todos los gráficos
cbar = fig.colorbar(cmatrix_plot[-1].im_, ax=ax.ravel().tolist())

# Establece el formateador para la barra de color para mostrar en porcentaje
cbar.ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))

# Añade un título general
fig.suptitle(f"Confusion Matrix for Each Mix Ratio of $CH_4$\n SNR={SNR}", fontsize=20)